# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
city_weather_df = pd.read_csv("output_data/cities.csv")
city_weather_df = city_weather_df.drop(columns="Unnamed: 0")


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [7]:
locations = city_weather_df[["Lat","Lng"]].astype(float)

humidity = city_weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#using loc with ideal weather conditions from HW formatting
city_weather_ideal = city_weather_df.loc[(city_weather_df['Max Temp'] > 70) & (city_weather_df['Max Temp'] < 80) & (city_weather_df['Cloudiness'] == 0) & (city_weather_df['Wind Speed'] < 10)]
city_weather_ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
30,Domna,51.90,113.16,78.80,39,0,2.24,RU,1592280473
220,Kiryat Gat,31.61,34.76,71.01,82,0,0.43,IL,1592280961
235,Andros,37.83,24.93,75.00,67,0,5.01,GR,1592281043
406,Boende,-0.22,20.87,71.94,80,0,0.25,CD,1592281072
422,Mbandaka,0.05,18.26,71.53,89,0,2.75,CD,1592281074
560,Soto la Marina,23.77,-98.22,79.00,87,0,1.01,MX,1592281094


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
hotel_df = pd.DataFrame(city_weather_ideal)
hotel_df.reset_index(drop=True, inplace = True)
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Domna,51.90,113.16,78.80,39,0,2.24,RU,1592280473,
1,Kiryat Gat,31.61,34.76,71.01,82,0,0.43,IL,1592280961,
2,Andros,37.83,24.93,75.00,67,0,5.01,GR,1592281043,
3,Boende,-0.22,20.87,71.94,80,0,0.25,CD,1592281072,
4,Mbandaka,0.05,18.26,71.53,89,0,2.75,CD,1592281074,
5,Soto la Marina,23.77,-98.22,79.00,87,0,1.01,MX,1592281094,


In [35]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key" : g_key}

#query for lodging/hotels within 5000 meter radius, if no hotel found within these parameters, populating DF with "No Hotel"
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat=row['Lat']
    lng=row['Lng']
    
    params["location"] = f"{lat},{lng}"
    
    hotel_query = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_query["results"][0]["name"]
    except (KeyError,IndexError):
        print('Missing field/result...populating "No Hotel".')
        hotel_df.loc[index, "Hotel Name"] = "No Hotel"
hotel_df

Missing field/result...populating "No Hotel".
Missing field/result...populating "No Hotel".


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Domna,51.90,113.16,78.80,39,0,2.24,RU,1592280473,No Hotel
1,Kiryat Gat,31.61,34.76,71.01,82,0,0.43,IL,1592280961,Desert Gat
2,Andros,37.83,24.93,75.00,67,0,5.01,GR,1592281043,Egli Boutique Hotel
3,Boende,-0.22,20.87,71.94,80,0,0.25,CD,1592281072,No Hotel
4,Mbandaka,0.05,18.26,71.53,89,0,2.75,CD,1592281074,Nina River Hotel
5,Soto la Marina,23.77,-98.22,79.00,87,0,1.01,MX,1592281094,Hacienda


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [37]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))